![Photo by Stephen Phillips - Hostreviews.co.uk on UnSplash](https://cf.bstatic.com/xdata/images/hotel/max1024x768/408003083.jpg?k=c49b5c4a2346b3ab002b9d1b22dbfb596cee523b53abef2550d0c92d0faf2d8b&o=&hp=1){fig-align="center" width=50%}


# Import data

In [1]:
import time
from pathlib import Path

import pandas as pd
from data import utils
from lets_plot import *
from lets_plot.mapping import as_discrete

LetsPlot.setup_html()

**Goal**:
- Identify what basic pre-processing steps need to be taken before uploading the data to a database

# Select Columns to Retain Based on the Quantity of Missing Values


In the realm of web scraping, managing the sheer volume of data is often the initial hurdle to conquer. It's not so much about deciding what data to collect but rather what data to retain. As we delve into the vast realm of the Imoweb website, we are met with a plethora of listings, each offering a unique set of information.

For many of these listings, there are commonalities – details like location and price tend to be constants. However, interspersed among them are those one-of-a-kind nuggets of information, such as the number of swimming pools available. While these specific details can certainly be vital in assessing the value of certain listings, the downside is that they can lead to a sparse dataset.

Currently, our primary objective is to pinpoint which features are prevalent across the board, drawing insights from a pre-scraped dataset comprising around 1000 ads. Once we've identified these common denominators, we can streamline our data collection process by retaining these key attributes while discarding the less likely occurrences.

In [31]:
for filename in utils.Configuration.RAW_DATA_PATH.glob("*.gzip"):
    if "data" in filename.stem:
        df = pd.read_parquet(filename)
df

,Address,Available as of,CO₂ emission,Covered parking spaces,Energy class,External reference,Flood zone type,Outdoor parking spaces,Planning permission obtained,Possible priority purchase right,...,Surroundings type,Furnished,Heating type,Bedroom 3 surface,Office,Latest land use designation,Street frontage width,"Gas, water & electricity",Price,Double glazing
0,Sint-Denijslaan 1 9000 - Gent,At delivery,Not specified,1,Not specified,5530472,Possible flood zone,1,Yes,No,...,None,NaN,None,None,NaN,None,None,None,None,None
1,Rue Simon 46/2 6990 - Hotton,Depending on the tenant,71 kg CO₂/m²,2,D,5530890,Property partially or completely located in a ...,None,Yes,No,...,None,NaN,None,None,NaN,None,None,None,None,None
2,Sint-Denijslaan 1 9000 - Gent,At delivery,Not specified,None,Not specified,5533819,Possible flood zone,None,Yes,None,...,None,NaN,None,None,NaN,None,None,None,None,None
3,Hoogstraat 20 9340 - Lede,After signing the deed,Not specified,None,F,5523589,Non flood zone,None,Yes,No,...,None,NaN,None,None,NaN,None,None,None,None,None
4,Heidestatiestraat 22 2920 - Kalmthout,None,Not specified,None,Not specified,95278 - NWB-23-SEM-0,Non flood zone,None,Yes,No,...,None,NaN,None,None,NaN,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Avenuedes Ardennes 7d 4500 - Huy,None,Not specified,1,E,18115 - 111115428,None,None,None,None,...,None,NaN,Fuel oil,None,NaN,None,None,No,"€ 265,000 265000 €",Yes
87,Sint-Denijslaan 1 9000 - Gent,None,Not specified,None,Not specified,5528536,Non flood zone,None,None,No,...,None,NaN,None,None,NaN,None,None,None,None,None
88,Avenue Louise 523 1050 - Ixelles,None,87 kg CO₂/m²,2,B,None,None,4,None,None,...,None,NaN,Gas,21 m² square meters,NaN,None,None,Yes,"€ 1,495,000 1495000 €",Yes
89,Avenue des Tournesols 14 1640 - Rhode-St-Genèse,None,37 kg CO₂/m²,None,D,5531996,None,None,None,None,...,None,NaN,Gas,16 m² square meters,NaN,None,5 m,Yes,"€ 445,000 445000 €",Yes


In [38]:
for columns in df:
    print(columns, df[columns].unique().shape)

Address (49,)
Available as of (5,)
CO₂ emission (32,)
Covered parking spaces (4,)
Energy class (8,)
External reference (85,)
Flood zone type (4,)
Outdoor parking spaces (6,)
Planning permission obtained (2,)
Possible priority purchase right (3,)
Primary energy consumption (67,)
Proceedings for breach of planning regulations (2,)
Reference number of the EPC report (60,)
Subdivision permit (3,)
Tenement building (2,)
Website (34,)
Yearly theoretical total energy consumption (20,)
ad_url (91,)
day_of_retrieval (91,)
Kitchen type (7,)
Width of the lot on the street (11,)
Kitchen surface (19,)
Dining room (2,)
Living area (43,)
Bedroom 1 surface (22,)
Basement (1,)
Connection to sewer network (2,)
As built plan (2,)
TV cable (2,)
Cadastral income (40,)
Bedrooms (8,)
Toilets (6,)
Building condition (6,)
Surface of the plot (50,)
Bedroom 2 surface (11,)
Garden surface (1,)
Construction year (42,)
Number of frontages (4,)
Bathrooms (5,)
Living room surface (26,)
Surroundings type (5,)
Furnishe

In [52]:
utils.Configuration.features_to_keep_sales

['day_of_retrieval',
 'ad_url',
 'Reference number of the EPC report',
 'Energy class',
 'Primary energy consumption',
 'Yearly theoretical total energy consumption',
 'CO₂ emission',
 'Tenement building',
 'Address',
 'Bedrooms',
 'Living area',
 'Bathrooms',
 'Surface of the plot',
 'Price',
 'Building condition',
 'Double glazing',
 'Number of frontages',
 'Website',
 'Toilets',
 'External reference',
 'Heating type',
 'Cadastral income',
 'Gas, water & electricity',
 'Latest land use designation',
 'Connection to sewer network',
 'Covered parking spaces',
 'Possible priority purchase right',
 'Proceedings for breach of planning regulations',
 'Construction year',
 'Subdivision permit',
 'Bedroom 1 surface',
 'Bedroom 2 surface',
 'Available as of',
 'Kitchen type',
 'Flood zone type',
 'Living room surface',
 'Planning permission obtained',
 'Kitchen surface',
 'TV cable',
 'Bedroom 3 surface',
 'Furnished',
 'Outdoor parking spaces',
 'Surroundings type',
 'Garden surface',
 'Base

In [81]:
(
    pd.read_csv(r"C:\Users\s0212777\individual_ad_22.csv")
    .set_index("0")
    .transpose()
    .filter(utils.Configuration.features_to_keep_sales)
    .pipe(
        lambda df: df.assign(
            **{
                col: pd.Series(dtype="float64")
                for col in set(utils.Configuration.features_to_keep_sales)
                - set(df.columns)
            }
        )
    )
)

0,Available as of,Construction year,Building condition,Street frontage width,Number of frontages,Surroundings type,Living area,Living room surface,Dining room,Kitchen type,...,Address,Website,External reference,day_of_retrieval,ad_url,Basement,Outdoor parking spaces,Planning permission obtained,Covered parking spaces,Proceedings for breach of planning regulations
1,To be defined,1918,Good,8 m,2,Isolated,133 m² square meters,32 m² square meters,Yes,Installed,...,Rue de la Wallonie 2A 4680 - Oupeye,http://www.nigel-immo.be,5529147,2023-09-22 20:42:43.891253,https://www.immoweb.be/en/classified/house/for...,NaN,NaN,NaN,NaN,NaN


In [85]:
csv_files = []
for i in Path(r"C:\Users\s0212777").glob("*.csv"):
    temp = pd.read_csv(i)
    csv_files.append(temp)

In [86]:
csv_files

[   Unnamed: 0         Available as of  Construction year Building condition  \
 0           1  After signing the deed               2016             As new   
 
   Street frontage width  Number of frontages  \
 0                   7 m                    3   
 
                            Surroundings type            Living area  \
 0  Living area (residential, urban or rural)  135  m² square meters   
 
     Living room surface Dining room  ... Kitchen surface  Office  \
 0  30  m² square meters         Yes  ...             NaN     NaN   
 
   Covered parking spaces Basement Toilets  Connection to sewer network  \
 0                    NaN      NaN     NaN                          NaN   
 
   Proceedings for breach of planning regulations Website  \
 0                                            NaN     NaN   
 
   Width of the lot on the street As built plan  
 0                            NaN           NaN  
 
 [1 rows x 51 columns],
    Unnamed: 0 Available as of  Construction year 

In [92]:
pd.concat(csv_files, axis=0)

,Unnamed: 0,Available as of,Construction year,Building condition,Street frontage width,Number of frontages,Surroundings type,Living area,Living room surface,Dining room,...,Kitchen surface,Office,Covered parking spaces,Basement,Toilets,Connection to sewer network,Proceedings for breach of planning regulations,Website,Width of the lot on the street,As built plan
0,1,After signing the deed,2016.0,As new,7 m,3.0,"Living area (residential, urban or rural)",135 m² square meters,30 m² square meters,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,To be defined,1918.0,Good,4 m,2.0,Isolated,105 m² square meters,14 m² square meters,Yes,...,12 m² square meters,NaN,NaN,Yes,1.0,NaN,NaN,http://www.penoit.be,NaN,NaN
0,1,To be defined,1900.0,Good,NaN,2.0,"Living area (residential, urban or rural)",130 m² square meters,24 m² square meters,Yes,...,18 m² square meters,NaN,NaN,NaN,3.0,Connected,NaN,http://www.mdi.be,NaN,NaN
0,1,At delivery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,http://www.immodavinci.be,NaN,NaN
0,1,NaN,1980.0,Good,NaN,4.0,NaN,744 m² square meters,NaN,NaN,...,NaN,NaN,16.0,NaN,5.0,NaN,NaN,https://agency.quares.be,31 m meters,No
0,1,To be defined,1953.0,Good,14 m,3.0,Urban,301 m² square meters,20 m² square meters,Yes,...,25 m² square meters,Yes,2.0,NaN,2.0,Not connected,No,http://www.living-stone.be,NaN,NaN
0,1,After signing the deed,1960.0,To be done up,6 m,3.0,"Living area (residential, urban or rural)",80 m² square meters,20 m² square meters,NaN,...,6 m² square meters,NaN,1.0,Yes,1.0,NaN,NaN,http://www.maxime-realestate.be,NaN,NaN
0,1,After signing the deed,1935.0,As new,NaN,3.0,Urban,90 m² square meters,21 m² square meters,NaN,...,6 m² square meters,NaN,NaN,NaN,2.0,Connected,NaN,http://www.amelotproperties.be,NaN,NaN
0,1,At delivery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No,http://www.immodavinci.be,NaN,NaN
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.heylenvastgoed.be,NaN,NaN


In [99]:
def immoweb_scraping_pipeline_short(
    last_page: int, start_page: int = 5
) -> pd.DataFrame:
    for page in range(start_page, last_page):
        print(page)


last_page = 10

immoweb_scraping_pipeline_short(last_page)

5
6
7
8
9
